# Time-Based Anomaly Analysis

This project explored time-based anomaly detection in credit card transactions by analyzing sudden spending spikes within short time frames. 

This approach is particularly useful for detecting:

- Bursts of high-value transactions in quick succession
- Unusual spending patterns that might indicate fraud
- Temporal clustering of transactions that could be suspicious

The method is simple but effective for identifying time-based anomalies where the total amount spent within a short period is unusually high compared to normal spending patterns.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

## Load Data

In [ ]:
df = pd.read_csv('creditcard.csv')

In [ ]:
print(df.head())
print(df.describe())

## Clean Data

In [ ]:
# Normalize numerical features

scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df.drop(columns=["Class"])), columns=df.columns[:-1])
df_scaled["Class"] = df["Class"].values  # Re-add fraud labels after scaling

In [ ]:
# Ensure 'Class' exists in the dataset
if "Class" not in df_scaled.columns:
    raise KeyError("Column 'Class' is missing from the dataset. Check column names!")

# Select only normal transactions (Class = 0)
normal_data = df_scaled[df_scaled["Class"] == 0].drop(columns=["Class"])

## Time-Based Anomaly Detection (Detect Spending Spikes)

**Time-Based Processing**
- Converts the 'Time' column to datetime format (assuming it represents seconds since epoch)
- Sorts the entire dataset by time to ensure chronological order

In [ ]:
# Convert 'Time' column to datetime format if it's in string format
df["Time"] = pd.to_datetime(df["Time"], unit='s')  # assuming Time is in seconds since epoch

# Sort by the 'Time' column
df = df.sort_values("Time")

**Rolling Sum Calculation**
- Creates a new column 'rolling_sum' that calculates the sum of transaction amounts over a rolling 5-transaction window
- This helps identify clusters of high-value transactions occurring close together in time

In [ ]:
# Rolling sum over a short time window to detect spikes
df["rolling_sum"] = df["Amount"].rolling(window=5, min_periods=1).sum()

**Anomaly Detection**
- Sets a threshold as the 99th percentile of rolling sums (top 1% of transaction clusters)
- Flags any rolling sums that exceed this threshold as potential anomalies ('spending_spike_anomaly')

In [ ]:
# Detect anomalies where rolling sum exceeds a threshold
threshold = df["rolling_sum"].quantile(0.99)  # Top 1% transactions are flagged
df["spending_spike_anomaly"] = (df["rolling_sum"] > threshold).astype(int)

## Visualize the results

- Plots the rolling sum over time
- Highlights the detected anomalies in red
- Creates a clear visual representation of spending patterns and spikes

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(df["Time"], df["rolling_sum"], label="Rolling Sum")
plt.scatter(df[df["spending_spike_anomaly"] == 1]["Time"], df[df["spending_spike_anomaly"] == 1]["rolling_sum"], color="red", label="Anomalies")
plt.xlabel("Time")
plt.ylabel("Rolling Sum of Transaction Amount")
plt.legend()
plt.title("Time-based Anomaly Detection: Spending Spikes")
plt.show()

## Conclusion

By calculating a rolling sum of transaction amounts over a 5-transaction window and flagging values exceeding the 99th percentile threshold, we successfully identified unusual spending patterns that could indicate potential fraud or anomalous behavior.

### Key Takeaways:
- Chronological Analysis Matters – Sorting transactions by time and using rolling calculations helps detect temporal anomalies that might be missed in static fraud detection methods.
- Simple Thresholds Can Be Effective – By comparing rolling sums against a high percentile (99%), we captured extreme spending clusters without complex modeling.
- Visualization Enhances Detection – The plotted rolling sums with highlighted anomalies provide an intuitive way to monitor transaction patterns over time.